In [3]:
# import tensorflow library
import tensorflow as tf
import time
# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
# one_hot key implies lables in onehot encoding
mnist = input_data.read_data_sets("dataset", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting dataset/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting dataset/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting dataset/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting dataset/t10k-labels-idx1-ubyte.gz


In [4]:
learning_rate = 0.001
training_iters = 20
batch_size = 5
display_step = 10

In [5]:
# Network Parameters
n_input = 784 # 28*28
n_classes = 10 # (0-9 digits)
dropout = 0.75 # What use?: to make network neuron more independent....

In [6]:
# define placeholders
x = tf.placeholder(tf.float32, shape=(None, n_input))
y = tf.placeholder(tf.float32, shape=(None, n_classes))

In [7]:
# miscill... functions
def define_variable(shape, name): 
    return tf.Variable(tf.truncated_normal(shape, name = name))

def conv2d(x, W, b):
    x = tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='VALID')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')


In [8]:
weights = {
    'wc1': define_variable([6, 6, 1, 40], 'wc1'), # 5x5 conv, 1 input, 40 outputs 
    'wc2': define_variable([6, 6, 40, 80], 'wc2'), # 5x5 conv, 40 inputs, 80 outputs
    'wc3': define_variable([3, 3, 80, 1024], 'wc3'), # 5x5 conv, 80 inputs, 500 outputs
    'out': define_variable([1024, n_classes], 'out') # 1024 inputs, 10 outputs (class prediction)
}
tf.summary.histogram("wc1", weights['wc1'])
tf.summary.histogram("wc2", weights['wc2'])
tf.summary.histogram("wc3", weights['wc3'])
tf.summary.histogram("out", weights['out'])

<tf.Tensor 'out_1:0' shape=() dtype=string>

In [9]:
biases = {
    'bc1': define_variable([40], 'bc1'),
    'bc2': define_variable([80], 'bc2'),
    'bc3': define_variable([1024], 'bc3'),
    'out': define_variable([n_classes], 'out')
}
tf.summary.histogram("bc1", biases['bc1'])
tf.summary.histogram("bc2", biases['bc2'])
tf.summary.histogram("bc3", biases['bc3'])
tf.summary.histogram("out", biases['out'])

<tf.Tensor 'out_3:0' shape=() dtype=string>

In [10]:
def main_net(x, weights, biases, dropout = dropout):
    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    
    conv1 = conv2d(x, weights['wc1'], biases['bc1']) # feature map dimension : 23x23
    pool1 = maxpool2d(conv1) # feature map dimension :  11x11

    conv2 = conv2d(pool1, weights['wc2'], biases['bc2']) # feature map dimension : 6x6
    pool2 = maxpool2d(conv2) # feature map dimension : 3x3
    
    conv3 = conv2d(pool2, weights['wc3'], biases['bc3']) # feature map dimension : 1x1
    
    unroll = tf.reshape(conv3, [-1, weights['out'].get_shape().as_list()[0]])
    print unroll
    
    out = tf.add(tf.matmul(unroll, weights['out']), biases['out'])
    return out

In [11]:
pred = main_net(x, weights, biases)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
merged = tf.summary.merge_all()
init = tf.global_variables_initializer()

Tensor("Reshape_1:0", shape=(?, 1024), dtype=float32)


In [12]:
with tf.Session() as sess:
    sess.run(init)
    writer = tf.summary.FileWriter('/tmp/data', sess.graph)
    step = 1
    while step * batch_size < training_iters:
        st = time.time()
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        if step % display_step == 0:
            # Calculate batch loss and accuracy
            summary, loss, acc = sess.run([merged, cost, accuracy], feed_dict={x: batch_x, y: batch_y})
            print("loss= {:.6f}".format(loss) + ", Accuracy= {:.5f}".format(acc))
        step += 1
    print("Optimization Finished!")
    print st
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={x: mnist.test.images[:256], y: mnist.test.labels[:256]}))

Optimization Finished!
1493893725.37
('Testing Accuracy:', 0.1015625)
